# Applied Data Science Capstone Project 

## This Project is for the ADS Capstone Project:

In [1]:
 pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


### Importing packages 

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Web-scrapping data 

#### Creating Beautiful Soup Object:

In [4]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table')
table_rows = table.tbody.find_all('tr')


#### Creating Pandas Data frame: 

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()

print("Shape:",df.shape)

Shape: (103, 3)


#### Importing Location Data:

As we are not able to use geocoder we are using the csv file provided 

In [7]:

df_geo_coor=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### We now need to pair the Postal Codes to their Lat/longs:

In [16]:
df_LL=pd.merge(df,df_geo_coor,left_on="PostalCode",right_on="Postal Code")
df_LL.head()



103

### Data Analysis:
Exploring and cluster the neighborhoods in Toronto.(Replicating the NewYork Analysis except now for Tornoto

#### Map of Toronto:
This is just a map of Tornonto and its Neighborhoods:

In [28]:

# Finding the mean point of all Neighborhoods so that Map is centered:
mean_lat=df_LL.Latitude.mean()
mean_long=df_LL.Longitude.mean()
print(mean_lat)
print(mean_long)

###
longitude = mean_long
latitude = mean_lat
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map based on coordinates of postcode
for lat, lng, Neighbourhood in zip(df_LL['Latitude'], df_LL['Longitude'], df_LL['Neighborhood']):
    label = '{}'.format(Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


43.70460773398059
-79.39715291165048


#### Surrounding areas:
This shows the top 5 venues in each Neigborhood of Toronto within a 500 meter radius.

In [18]:
# Log into Foursquare:

CLIENT_ID = 'BQCYU4IEPQXQ005Y4HAYT4QWPWRKBTKGLFRNV4HSKFMPIZDN' # your Foursquare ID
CLIENT_SECRET = '0K12WQKYXZB3PD4SMM2DORTYKOKOIU34ZTW3EQF0LN3EG4Q3' # your Foursquare Secret
VERSION = '20161225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BQCYU4IEPQXQ005Y4HAYT4QWPWRKBTKGLFRNV4HSKFMPIZDN
CLIENT_SECRET:0K12WQKYXZB3PD4SMM2DORTYKOKOIU34ZTW3EQF0LN3EG4Q3


In [19]:
# define inputs:
# Number of locations to be displayed:
Locations = 5
# Distance from neighbourhood to be included:
distance = 500

locations = [] # initiate a list to store data from Foursquare API requests

for neighbourhood, latitude, longitude in zip(df_LL.Neighborhood, df_LL.Latitude, df_LL.Longitude):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    distance, 
    Locations)
    
    data = requests.get(url).json()
    
    # use len() to check if any data within "items" (len = 0 if nothing)
    length = len(data['response']['groups'][0]['items'])
    if length == 0:
        continue # skip the row if nothing is found
    else:
        venue = data['response']['groups'][0]['items'][0]['venue']
        
        # extract info within 'venue'
        name = venue['name']
        lat = venue['location']['lat']
        lon = venue['location']['lng']
        cat = venue['categories'][0]['name']
        
        locations.append([(neighbourhood, latitude, longitude, name, lat, lon, cat)])

In [41]:
temp = pd.DataFrame(x for row in locations for x in row)
temp.columns = ['Neighborhood','N_Latitude','N_Longitude','Venue','V_Latitude','V_Longitude','category']
temp.head()
print("{} nearby locations downloaded for {} neighborhood.".format(len(temp.Venue), len(df_LL.Neighborhood)))

98 nearby locations downloaded for 103 neighborhood.


We now need to find the frequency of each category around each location: 
Using get_dummies to create a matrix of location and category freqency

In [55]:
category_freqency = pd.get_dummies(temp.category) # one hot encoding with get_dummies()
Neigh_Cat_Freq = pd.concat([temp[['Neighborhood']], category_freqency], axis=1) # combine neighbourhood & category tables
Neigh_Cat_Freq.head()

,Neighborhood,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Clustering Neigborhoods:
We will now cluster the Toronto Neighborhoods:

In [44]:
Neigh_Cat_Freq=Neigh_Cat_Freq.squeeze()
print(type(Neigh_Cat_Freq['Neighborhood']),type(df_LL['Neighborhood']))
print(len(Neigh_Cat_Freq['Neighborhood'].unique()),len(df_LL['Neighborhood'].unique()))


<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


AttributeError: 'DataFrame' object has no attribute 'unique'

In [83]:
# Importing the nescasery packages:
from sklearn.cluster import KMeans


In [54]:
Neigh_Cat_Freq=Neigh_Cat_Freq['Neighborhood'].ix[:,0]
print(type(Neigh_Cat_Freq['Neighborhood']),type(df_LL['Neighborhood']))

AttributeError: 'DataFrame' object has no attribute 'ix'

In [58]:
#Inputs:
## The input for kMeans needs to be a matrix, we therefore need to remove the first column that contains the Neighborhoods name.
Neigh_Cat_Freq2 = Neigh_Cat_Freq.drop('Neighborhood', axis=1)

## Number of clusters:
k=6

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(Neigh_Cat_Freq2)
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 5, 0, 2], dtype=int32)

In [59]:
# Clustering labels
Neigh_Cat_Freq.insert(1, 'label', kmeans.labels_)

In [63]:
Neigh_Cat_Freq.head(10)

,Neighborhood,label,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Scarborough Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Kennedy Park, Ionview, East Birchmount Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Golden Mile, Clairlea, Oakridge",5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Cliffside, Cliffcrest, Scarborough Village West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Birch Cliff, Cliffside West",2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
 # Removing duplicate column headings:
df1 = Neigh_Cat_Freq.loc[:, ~Neigh_Cat_Freq.columns.duplicated()]
    #Combining the geolocation data(df_LL) with category data (Neigh_Cat_Freq)
toronto_combined=pd.merge(df_LL,df1, on ='Neighborhood',how ='left' )
toronto_combined=toronto_combined.fillna(0)

In [93]:
toronto_combined.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,label,Airport,Arts & Crafts Store,Bakery,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint,Yoga Studio
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_combined['Latitude'], toronto_combined['Longitude'], toronto_combined['Neighborhood'], toronto_combined['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[(int(cluster))-1],
        fill=True,
        fill_color=rainbow[(int(cluster))-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters